# Kontekstualizovano ugnježdavanje reči

### Uvod

**Kontekstualizovano ugnježdavanje reči (Contextualized Word Embeddings, CWE)** je postupak formiranja novih reprezentacija reči u zavisnosti od konteksta u kome se reči nalaze. Ideja je pridružiti rečima vektorske reprezantcija takve da je uzet u obzir i kontekst. Na taj način se obuhvata upotreba reči u različitim kotekstima i to znanje ugrađuje u vektorsku reprezentaciju. Formalno, reprezentacije reči odnosno tokena su funkcije od ulazne rečenice. Na primer, reč "Vašington" u rečenici "Univerzitet u Vašingtonu" treba biti protunačena kao naziv ustanove, jer se u tom kontekstu i koristi, a ne kao lično ime ili naziv grada.

Pre razvoja CWE tehnika uglavnom se pristupalo pridruživanju jednog globalnog značenja reči među ostalim rečima u velikim korpusima teksta pomoću modela nenadgledanog učenja:
- Joseph Turian, Lev Ratinov, and Yoshua Bengio. 2010. "Word representations: a simple and general method for semi-supervised learning", Koferencija o empirijskim metodama i obradi prirodnih jezika 2014.
- Tomas Mikolov, Kai Chen, Greg Corrado, and Jeffrey Dean. 2013. "Efficient estimation of word representations in vector space"
- Jeffrey Pennington, Richard Socher, and Christopher Manning. 2014. "Glove: Global vectors for word representation", Koferencija o empirijskim metodama i obradi prirodnih jezika 2014.

Ovi radovi nisu uzimali u obzir kontekst u kome se reč nalazi. Jedan od prvih radova koji je uveo kontekst u reporezentacije reči bio je "Semi-supervised sequence tagging with bidirectional language models", Matthew E. Peters, Waleed Ammar, Chandra Bhagavatula, Russell Power 2017. Smatra se pretečom modernih CWE modela, koji danas važe za poslednju reč tehike (state-of-the-art). Među njima su radovi:
- Matthew E Peters, Mark Neumann, Mohit Iyyer, Matt Gardner, Christopher Clark, Kenton Lee, and Luke Zettlemoyer. 2018. "Deep contextualized word repre-sentations", poznatiji kao ELMO model
- Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. 2018. "Bert: Pre-training of deep bidirectional transformers for language understanding"
- Zhilin Yang, Zihang Dai, Yiming Yang, Jaime Carbonell, Ruslan Salakhutdinov, and Quoc V Le. 2019. "Xlnet: Generalized autoregressive pretraining for language understanding"
- Colin Raffel, Noam Shazeer, Adam Roberts, Katherine Lee, Sharan Narang, Michael Matena, Yanqi Zhou, Wei Li, and Peter J Liu. 2019. "Exploring the limits of transfer learning with a unified text-to-text transformer"

Modeli predstavljni u ovim radovima pokazali su da dobijene CWE reprezentacije reči postižu izuzetne performanse na različitim zadacima obrade prirodnih jezika kao što su klasifikacija teksta, odgovaranje na pitanja, rezimiranje teksta.

### Sadržaj projekta
- U uvodnoj (ovoj) svesci biće predstavljen "*Contextual String Embeddings for Sequence Labeling*" Alan Akbik, Duncan Blythe i Roland Vollgraf 2020. i CWE reprezentacije po nazivom "*Contextual string embeddings*" predstavljen u radu. Nučnici su sve svoje modele i kod organizovali u *[Flair](https://github.com/flairNLP/flair)* radni okvir koji je javno dostupan. Biće predstavljen3 i neke od osnovnih funkcionalnosti Flair radnog okvira.

- U drugoj svesci biće predstavljen način za učitavanje skupova podataka koje Flair nudi kao i opšti postupak treniranje modela za prepoznavanje *upos* etiketa uz korišćenje CWE modela iz radnog okvira. Model je treniran na skupu rečenica na srpskom jeziku i rezltati su prikazani na kraju sveske.

- U trećoj svesci predstavljen je postupak formiranja CWE modela nad proizvoljnim korupuson. Korišćeni su tekstovi na srpsom jeziku. Model je dalje iskorišćem za treniranje modela za prepoznavanje *upos* etiketa i rezultati su prikazani na kraju sveske

### "Contextual String Embeddings for Sequence Labeling" Alan Akbik, Duncan Blythe i Roland Vollgraf 2020.

Autori su u radu predstavili model za formiranje CWE reprezentacija reči. Reprezentacije koje model daje su nazvali "*Contextual string embeddings*". Glavna karakteristika ovog jezičkog model je to što je on formiran na nivou pojedinačnih karaktera u rečenici. To znači da se za kotekst jedne reči uzima u obzir kompletna sekvenca karaktera ispred i iza nje. Ova karakteristika izdvaja model od ostalih koji pri formiranju konteksta u kome se reč nalazi uzimaju u obzi samo okolne reči a ne cele sekvence karaktera.

#### Arhitektura modela

Za formiranje "Contextual string embeddings" reprezentacija autori su iskoristili LSTM varijanu rekurentnih neuronskih mreža za koju navode da se pokazala kao bolji pristup pri formiranju jezičkih modela u donsu na n-grame, pozivajući se na rezultate iz ranijih radova. Razlog tome je činjenica da LSTM neuralne mreže mogu fleksibilno da modeluju dugoročne zavisnosti sa svojim skrivenim stanje. Kao što je već rečeno, osnovna grdivna jedinica modela je karakter što za posledicu ima to da se rečenica tretira kao sekvenca karaktera prosleđena LSTM-u, a sam LSTM je treniran da u svakoj tački te sekvence predviđa sledeći karakter.

Formalno, cilj jezičkog modela zasnovanog na karakterima je da dobro modeluje raspodelu niza karaktera $P(x_{0:T})$, gde je $(x_0, x_1, ..., x_T):=x_{0:T}$. Treniranjem modela uči se uslovna raspodela karaktera $x_t$ pri datom nizu karaktera, odnosno $P(x_t|x_0,...,x_{t-1})$. Tada se raspodela cele rečenice (čitavog niza karaktera) može dekomponovati na prozvod uslovnih raspodela pojedinačnih karaktera pri nizu karaktera koji im prethode, odnosno 

$$P(x_{0:T}) = \prod_{t=0}^{T}{P(x_t|x_{0:t-1})}$$

U LSTM mreži uslovna verovatnoća $P(x_t|x_{0:t-1})$ se aproksimira funkcijom od izlaza iz neuronse mreže $\boldsymbol{h_t}$, odnosno 

$$P(x_t|x_{0:t-1}) \approx \prod_{t=0}^{T}{P(x_t|\boldsymbol{h_t};\theta})$$

$\boldsymbol{h_t}$ predstavlja čitavu prošlost sekvence karaktera. U LSTM-u se računa rekurzivno, uz dodatnu pomoć memorijske ćelije $c_t$

$$\boldsymbol{h_t}(x_{0:t-1}) = f_h(x_{t-1}, \boldsymbol{h_{t-1}}, c_{t-1}; \theta)$$

$$c_t(x_{0:t-1}) = f_c(x_{t-1}, \boldsymbol{h_{t-1}}, c_{t-1}; \theta)$$ 

gde $\theta$ označava hiperparametre modela a $\boldsymbol{h_{-1}}$ i $c_{-1}$ se mogu inicijalizovati na nulu ili tretirati kao deo hiperparametara $\theta$.

Na izlaz iz mreže $\boldsymbol{h_t}$ je postavljen potpuno povezn sloj sa *softmax* aktivacionom funkcijom, tako da je verovatnoća svakog karaktera data sa

$$P(x_t|\boldsymbol{h_t};V)=softmax(V\boldsymbol{h_t} + b)$$

gde su težine $V$ i slobodni članovi $b$ deo hiperparametara modela.

Zbog karakteristike ovakve rekurentne neuronske mreže, da se informaciju u skrivenim stanjima šalju u oba smera, uz gore opisani model unapred može se dobiti i model unazd (*eng. backward model*, odakle i dolazi $b$ u eksponentu u formulama ispod), odnosno

$$P^b(x_t|x_{t+1:T}) \approx \prod_{t=0}^{T}{P^b(x_t|\boldsymbol{h_t^b};\theta})$$

$$\boldsymbol{h_t^b} = f_h^b(x_{t+1}, \boldsymbol{h_{t+1}^b}, c_{t+1}^b; \theta)$$

$$c_t^b = f_c^b(x_{t+1}, \boldsymbol{h_{t+1}}^b, c_{t+1}^b; \theta)$$

Model unapred i model unazad se koriste na sledći način kako bi se formirala "*Contextual string embeddings*" reprezentacija za neku reč:
- Iz modela unapred uzima se izlaz skrivenog sloja nakon poslednjeg karaktera u željenoj reči. Kako je model unapred treniran da predvidi sledći karakter u sekvenci, skriveni sloj do tog trenutka nosi kontekst od početka rečenive sve do odabrane reči, uključujuću i tu reč (smer unapred).
- Slično, iz modela unazad uzima se izlaz skrivenog sloja pre prvog karaktera u željenoj reči. Kako je model unazad treniran da predvidi prethodni karakter u sekvenici, skriveni sloj do tog trenutka nosi kontekst od kraja rečenice sve do odabrane reči, uljučujuči i tu reč (smer unazad).
- Ova dva izlaza se zatim nadovežu kako bi se formirao "*Contextual string embeddings*" za željenu reč koji će nositi informaciju o samoj reči ali i informaciju o okolnom kontekstu

Formalno, za reč čiji su karakteri $t_0, t_1,..., t_n$ "*Contextual string embeddings*" se formira kao
$$w_i^{charLM} := \begin{bmatrix} h^f_{t_{i+1}-1} \\ h^b_{t_{i}-1} \end{bmatrix}$$

Opisani postupak je ilustrovan na slici ispod
![Formiranje CWE](resources/images/cwe_scheme.PNG)

Ovako dobijene "*Contextual string embeddings*" vektorske reprezentacije reči se mogu koristit pri rešavanju različitih problema obrade prirodnih jezika. Način korišćenja je ilustrovan na slici ispod

![Koriscenje CWE](resources/images/cwe_usage.PNG)

### Flair

TODO